In [1]:
from vantage6.client import Client

In [42]:
client = Client("http://localhost", 5000, "/api")
client.authenticate("node1-admin", "password")
client.setup_encryption(None)

In [43]:
# Find out the organizations and collaborations we can run on
import json
collaboration_list = client.collaboration.list()
collaboration_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[collaboration_index]['organizations']:
    organization_ids_.append(organization['id'])
print(json.dumps(client.node.list(), indent=2))

[
  {
    "name": "Alpha Hospital - SWAT4LS2022 Node",
    "collaboration": {
      "id": 1,
      "link": "/api/collaboration/1",
      "methods": [
        "GET",
        "PATCH",
        "DELETE"
      ]
    },
    "type": "node",
    "organization": {
      "id": 3,
      "link": "/api/organization/3",
      "methods": [
        "PATCH",
        "GET"
      ]
    },
    "status": "online",
    "id": 5,
    "ip": "169.254.130.1",
    "last_seen": "2022-01-09T11:51:32.016352+00:00",
    "api_key": "7906e5ce-6d70-11ec-8397-726d2853664e"
  }
]


In [44]:
# Write your query here
query = """
prefix snomedct: <http://purl.bioontology.org/ontology/SNOMEDCT/> 
prefix roo: <http://www.cancerdata.org/roo/>
prefix ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>


select ?survival_5y ?lymphNodesExamined ?lymphNodesPositive ?estrogenReceptor ?progresteronReceptor ?her2Receptor ?tumorSize ?clinTStage ?clinNStage ?clinMStage ?pathTStage ?pathNStage ?axNodeDissection

where { 
        ?patient a snomedct:116154003;
                ncit:P395 ?patientIdentifier. #targetCode
    	?patient roo:survival_5_years ?survival_5y.
		?patient roo:P100008 ?tumour.#has neoplasm
   		?tumour ncit:P395 ?diseaseEpisode;	
             	roo:P100042 ?tumourTypeClass; 
             	roo:P100258 [ #has_disease_finding
        				rdf:type ncit:C124446;
          				roo:P100042 ?lymphNodesExamined].
optional{?tumour ncit:P395 ?diseaseEpisode;	
            	roo:P100258 [ #has_disease_finding
            		rdf:type ncit:C124447;
             		roo:P100042 ?lymphNodesPositive].}          
optional{?tumour ncit:P395 ?diseaseEpisode;	
                 roo:P100258 [ #has_disease_finding
        			rdf:type ncit:C16150;	
          			roo:P100042 ?estrogenReceptor].}
optional{?tumour ncit:P395 ?diseaseEpisode;	    
          	 	roo:P100258 [ #has_disease_finding
        			rdf:type ncit:C16149;	
          			roo:P100042 ?progresteronReceptor].}
optional{?tumour ncit:P395 ?diseaseEpisode;	        
          	 	roo:P100258 [ #has_disease_finding
        			rdf:type ncit:C68748;	
           			roo:P100042 ?her2Receptor].}        
optional{?tumour ncit:P395 ?diseaseEpisode;	    
    			roo:P100214 [ #has tumour size	
            	roo:P100042 ?tumorSize].}
optional{?tumour ncit:P395 ?diseaseEpisode;	        
          	 	roo:P100243[ #has clinical stage
        			roo:P100244 [ #has clinical t stage               		
            		roo:P100042 ?clinTStage]].}
optional{?tumour ncit:P395 ?diseaseEpisode;	  
                roo:P100243[ #has clinical stage
                 	roo:P100242 [ #has clinical n stage
            		roo:P100042 ?clinNStage]].}
optional{?tumour ncit:P395 ?diseaseEpisode;	     					
           		roo:P100243[ #has clinical stage	
                     roo:P100241 [ #has clinical m stage
            		 roo:P100042 ?clinMStage]].}  
optional{?tumour ncit:P395 ?diseaseEpisode;	             
          		roo:P100243[ #has clinical stage
                 	roo:P100288 [ #has path t stage
            		roo:P100042 ?pathTStage]].}
optional{?tumour ncit:P395 ?diseaseEpisode;	             
          		roo:P100243[ #has clinical stage               
            		roo:P100286 [ #has path n stage
            		roo:P100042 ?pathNStage]].}
optional{?tumour ncit:P395 ?diseaseEpisode;	             
          		roo:P100243[ #has clinical stage    
					roo:P100285 [ #has path m stage
           			roo:P100042 ?pathMStage]]. }
optional{?tumour ncit:P395 ?diseaseEpisode;	             
          		roo:P100024[ #has procedure
            		rdf:type ncit:C15677;
              		roo:P100042 ?axNodeDissection].}
    
}
"""

In [45]:
# Request the the model to be trained and evaluated
input_ = {
    "master": "true",
    "method":"master", 
    "args": [ ],
    "kwargs": {
        "query": query,
        "dependent_var": "survival_5y",
    }
}

task = client.post_task(
    name="Model",
    image="jaspersnel/v6-svr-centralized-py-rdf:test06",
    collaboration_id=collaboration_list[collaboration_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]]
)

print(json.dumps(task, indent=2))

{
  "name": "Model",
  "collaboration": {
    "id": 1,
    "link": "/api/collaboration/1",
    "methods": [
      "GET",
      "PATCH",
      "DELETE"
    ]
  },
  "image": "jaspersnel/v6-svr-centralized-py-rdf:test06",
  "results": [
    {
      "id": 186,
      "link": "/api/result/186",
      "methods": [
        "PATCH",
        "GET"
      ]
    }
  ],
  "description": "",
  "children": null,
  "id": 106,
  "initiator": 3,
  "complete": false,
  "run_id": 59,
  "parent": null,
  "database": ""
}


In [46]:
# Wait for results
import time
import json
resultObjRef = task.get("results")[0]
resultObj = client.result.get(resultObjRef['id'])
attempts = 1
while((resultObj["finished_at"] == None) and attempts < 10):
    print("waiting...")
    time.sleep(5)
    resultObj = client.result.get(resultObjRef['id'])
    attempts += 1
results = resultObj['result']

print(results)

waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
waiting...
{'r_squared': 0.7128245476003148}


In [47]:
print(resultObj['log'])

{'log': 'info > wrapper for v6_svr_centralized_py\ninfo > Reading input file /mnt/data/task-000000186/input\ninfo > No data format specified. Assuming input data is pickle format\ninfo > Reading token file \'/mnt/data/task-000000186/token\'\ninfo > Using \'http://graphdb:7200/repositories/hospital_alpha\' as database\ninfo > Dispatching ...\ninfo > Module \'v6_svr_centralized_py\' imported!\ninfo > Running a master-container\ninfo > Working with collaboration_id <1>\ninfo > Defining input parameters\ninfo > Dispatching training task to one node\ninfo > Waiting for resuls\ninfo > Waiting for results\ninfo > Waiting for results\ninfo > Waiting for results\ninfo > Waiting for results\ninfo > Waiting for results\ninfo > Obtaining trained model\ninfo > [{\'model\': b\'\\x80\\x03csklearn.pipeline\\nPipeline\\nq\\x00)\\x81q\\x01}q\\x02(X\\x05\\x00\\x00\\x00stepsq\\x03]q\\x04(X\\x11\\x00\\x00\\x00columntransformerq\\x05csklearn.compose._column_transformer\\nColumnTransformer\\nq\\x06)\\x81q\\x